# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [107]:
import pandas as pd
import numpy as np
from scipy import stats

In [108]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [109]:
# number of callbacks for black-sounding names
black_calls_array=data[data.race=='b']['call'].values
white_calls_array=data[data.race=='w']['call'].values

In [110]:
data.shape

(4870, 65)

In [171]:
#mean and std dvn for binomial distribution for black interview calls
mu_black=np.mean(black_calls_array)
sigma_black=np.std(black_calls_array)
print('frac black calls=', mu_black)
print('std dvn black calls=',sigma_black)

#mean and std dvn for binomial distribution for white interview calls
mu_white=np.mean(white_calls_array)
sigma_white=np.std(white_calls_array)
print('frac white calls=', mu_white)
print('std dvn white calls=',sigma_white)

#mean and st dvn for combined data
comb_array=np.concatenate((white_calls_array, black_calls_array))
mu_combined=np.mean(comb_array)
sigma_combined=np.std(comb_array)
print('frac all calls=', mu_combined)
print('std dvn all calls=',sigma_combined)

#diff between white and black call rates
diff=mu_white-mu_black
print('diff in call rates=',diff)

frac black calls= 0.0644764
std dvn black calls= 0.2456
frac white calls= 0.0965092
std dvn white calls= 0.295288
frac all calls= 0.0804928
std dvn all calls= 0.272055
diff in call rates= 0.0320329


In [112]:
#lenght of black and white job applicant arrays
len_black=len(data[(data['race']=='b')])
len_white=len(data[(data['race']=='w')])
print(len_black)
print(len_white)
#rechecking fraction of blacks and whites getting interview calls
black_calls_frac=black_calls/len_black
white_calls_frac=white_calls/len_white
print('fraction black interview=',black_calls_frac)
print('fraction wnite interview=',white_calls_frac)

2435
2435
fraction black interview= 0.064476386037
fraction wnite interview= 0.0965092402464


# Q1 What test is appropriate for this problem? Does CLT apply?

Ans1 : As per CLT , the sample means of any random variable drawn from a population are normally distributed. What would be normally distributed here is the sample mean of fraction of blacks or whites getting a interview call. We can do permutation test to check if fraction of blacks or whites called for interview is essentially is the same. We will also do z and t test.

# Q2 What are the null and alternate hypotheses?

Ans2: The null hypothesis is that the the  black callback rate and white call rate is essentially the same. Alternate hypothesis is that the black callback rate is less than white call back rate.

# Q3 Compute margin of error, confidence interval, and p-value.

Ans3: We will do 
    1)permutations test
    2) t test
    3) ztest

In [162]:
#function to draw permutation samples
def draw_perm_reps(data1, data2, size=1):
    """Generate multiple permutation replicates."""
    #size indicates number os amples
    # Initialize array of replicates: perm_replicates
    perm_replicates = np.empty(size)
    combined_data = np.concatenate((data1, data2))

    for i in range(size):
        # Generate permutation sample
        permuted_data = np.random.permutation(combined_data)
        perm_sample_1 = permuted_data[:len(data1)]
        perm_sample_2 = permuted_data[len(data2):]
        # Compute the test statistic
        perm_replicates[i] = np.mean(perm_sample_1)-np.mean(perm_sample_2)

    return perm_replicates

In [175]:
perm_replicate_calls=draw_perm_reps(white_calls_array, black_calls_array, 1000)
p = np.sum(perm_replicate_calls >= diff) / len(perm_replicate_calls)

# Print the result
print('p-value =', p)


p-value = 0.0


In [176]:
print(np.mean(perm_replicate_calls))
print(np.std(perm_replicate_calls))

-0.000202053278685
0.00794524766859


In [177]:
conf_int = np.percentile(perm_replicate_calls, [2.5, 97.5])
s1,s2=(diff+conf_int[0],diff+conf_int[1])
print('95% confidence interval is (',s1,'  ',s2,')')

95% confidence interval is ( 0.0164065659046    0.0476386100054 )


In [166]:
#t test
stats.ttest_ind(white_calls_array,black_calls_array,equal_var=False)

Ttest_indResult(statistic=4.1147052908617514, pvalue=3.9429415136459352e-05)

In [184]:
 #z test
denom=(np.sqrt(mu_combined*(1-mu_combined)*(1/len_black+1/len_white)))
z_stat=(mu_white-mu_black)/denom
print('z stat is ',z_stat)
pval = 2*(1 - stats.norm.cdf(np.abs(z_stat)))
print('p stat is',pval)
#95% confidence interval
print('confidence interval at 95% is (',diff-1.96*denom,' ',diff+1.96*denom,')')
print('margin of error is +/-',1.96*denom)


z stat is  4.10841214853
p stat is 3.98388690499e-05
confidence interval at 95% is ( 0.0167509422216   0.0473147674885 )
margin of error is +/- 0.0152819126334


Permutation, t and z test shows that the null hypothesis that inteview callback rates for blacks and whites are the same can be rejected. At 95% confidence level, the margin of error +/- +0.015.

# Q4 Write a story describing the statistical significance in the context or the original problem.


Resumes with black-sounding names have interview call back rate about 3.2% lower than white-sounding names. The null hypothesis that the callback rates is essentially the same was rejected. The p-value for the test of equality of callback rates is of the order of 5th decimal place. The 95% confidence level of difference between white and black call back rates was (1.6%,4.7%).

# Q5 Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

Our analysis shows that the interview callback rates for white-sounding and black-sounding names is not the same. It does not conclude that race is the most important factor in getting interview calls. We will need to test other features in the data for their impact on interview call back rates. We would also need to check for correlation between race and other features.